 # Preparación y Limpieza de Datos — Incendios en España

 Este cuaderno prepara los datos utilizados por el *Spain Wildfires Dashboard*:
 - Carga el CSV y el GeoJSON
 - Selecciona y tipa columnas
 - Crea la bandera **intencionado**
 - Mapea `idprovincia` → `provincia`
 - Exporta un dataset limpio para análisis y visualizaciones

 **Autores:** Lorena Elena Mohanu · José Ancízar Arbeláez Nieto
 **Licencia:** MIT (ver `LICENSE` en el repositorio)

 ## Importaciones

In [17]:
# --- Configuración robusta de rutas para importar `wildfires` desde notebooks/ ---
from pathlib import Path
import sys

cwd = Path.cwd().resolve()
candidates = [cwd, cwd.parent, cwd.parent.parent]  # ., .., ../..
added = False

for base in candidates:
    src = base / "src"
    if (src / "wildfires").exists():
        sys.path.insert(0, str(src))
        added = True
        break

if not added:
    raise RuntimeError(
        "No se encontró 'src/wildfires' en ., .. o ../..  "
        "Asegúrate de ejecutar el notebook dentro del repo."
    )

# Prueba de import
import wildfires
print("wildfires importado desde:", wildfires.__file__)

wildfires importado desde: C:\Users\loren\Downloads\Semestre 5\visualizacion_de_datos\vdat-p3-GJ01\Proyecto_visualizacion_incendios\src\wildfires\__init__.py


In [18]:
import pandas as pd

# `display` para notebooks; si ejecutas como script, lo redefinimos a print
try:
    from IPython.display import display
except Exception:  # pragma: no cover
    def display(x):  # type: ignore
        print(x)

import sys, os
sys.path.append(os.path.abspath("src"))

from wildfires import (
    read_wildfires_csv, read_geojson, write_parquet,
    prepare_wildfires,
    build_provinces_map, map_province_names
)

 ## Rutas de datos

In [19]:
# Ajusta estas rutas a tu estructura
CSV_PATH = "../data/raw/incendios.csv"
GEOJSON_PATH = "../data/raw/spain-provinces.geojson"
OUTPUT_PARQUET = "../data/processed/incendios_clean.parquet"

 ## Carga de datos

In [20]:
df_raw = read_wildfires_csv(CSV_PATH, sep=";")
geojson = read_geojson(GEOJSON_PATH)

print("Filas crudas:", len(df_raw))
display(df_raw.head(3))

C:\Users\loren\Downloads\Semestre 5\visualizacion_de_datos\vdat-p3-GJ01\Proyecto_visualizacion_incendios\src\wildfires\io.py:38: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=sep, encoding=encoding, **read_csv_kwargs)


Filas crudas: 585399


,idpif,anio,idestadopif,probabilidadignicion,idpeligro,idcomunidad,comunidad,idprovincia,provincia,idmunicipio,...,numeromediosaereos,huso,x,y,iddatum,claseincendio,lugar,combustible,tipodefuego,tipodeataque
0,568679.0,2016,3,NaN,NaN,15,Islas Baleares,7,Islas Baleares,63,...,0,31.0,466451.0,4397989.0,2.0,conato,excursionistas,bosques,superficie,directo
1,568680.0,2016,3,NaN,NaN,10,Aragon,22,Huesca,157,...,0,30.0,800159.0,4713460.0,2.0,conato,vertederos,pas_mat,superficie,directo
2,568681.0,2016,3,NaN,NaN,10,Aragon,22,Huesca,125,...,0,30.0,714747.0,4675882.0,2.0,conato,NaN,pastizal,superficie,directo


 ## Preparación: selección, tipos, bandera `intencionado`, alias de hectáreas

In [21]:
# Aplica el pipeline mínimo de limpieza y preparación
df_prepared = prepare_wildfires(df_raw)

display(df_prepared.head(5))
print("\nTipos de datos:")
print(df_prepared.dtypes)

,anio,idpeligro,idprovincia,provincia,numeromediospersonal,numeromediospesados,numeromediosaereos,perdidassuperficiales,idcausa,intencionado,hectareas_quemadas
0,2016,-1,7,Islas Baleares,20,18,0,0.20,290,False,0.20
1,2016,-1,22,Huesca,4,0,0,0.05,322,False,0.05
2,2016,-1,22,Huesca,9,4,0,0.23,241,False,0.23
3,2016,-1,22,Huesca,8,4,0,0.65,212,False,0.65
4,2016,-1,22,Huesca,7,3,0,0.15,212,False,0.15



Tipos de datos:
anio                       int64
idpeligro                  int32
idprovincia                int32
provincia                 object
numeromediospersonal       int64
numeromediospesados        int64
numeromediosaereos         int64
perdidassuperficiales    float64
idcausa                    int64
intencionado                bool
hectareas_quemadas       float64
dtype: object


 ## Mapeo de provincias

In [22]:
prov_map = build_provinces_map(geojson)
df = map_province_names(df_prepared, prov_map)

# Resumen de años disponibles (si existe la columna 'anio')
years = sorted(df["anio"].dropna().astype(int).unique().tolist()) if "anio" in df.columns else []
print("\nAños disponibles:", years[:5], "...", years[-5:] if years else [])
display(df.head(5))


Años disponibles: [1968, 1969, 1970, 1971, 1972] ... [2012, 2013, 2014, 2015, 2016]


,anio,idpeligro,idprovincia,provincia,numeromediospersonal,numeromediospesados,numeromediosaereos,perdidassuperficiales,idcausa,intencionado,hectareas_quemadas
0,2016,-1,7,Illes Balears,20,18,0,0.20,290,False,0.20
1,2016,-1,22,Huesca,4,0,0,0.05,322,False,0.05
2,2016,-1,22,Huesca,9,4,0,0.23,241,False,0.23
3,2016,-1,22,Huesca,8,4,0,0.65,212,False,0.65
4,2016,-1,22,Huesca,7,3,0,0.15,212,False,0.15


 ## Exportación a formato eficiente (Parquet)

In [23]:
# Creamos la carpeta de salida si no existe
os.makedirs(os.path.dirname(OUTPUT_PARQUET), exist_ok=True)

write_parquet(df, OUTPUT_PARQUET)
print(f"\nGuardado: {OUTPUT_PARQUET}")


Guardado: ../data/processed/incendios_clean.parquet
